# Lidarデータの処理(pdal pilepine 編)

In [56]:
from io import BytesIO
import json

import geopandas as gpd
from IPython.display import IFrame
import k3d
import laspy
import numpy
from matplotlib import pyplot as plt
import pdal
import shapely
import rich


input_las = "../datasets/01ID7913_org.las"

In [50]:
# url = 'https://www.keene.edu/campus/maps/tool/'
# IFrame(url, width=1200, height=500)

## データの概要を確認する。
---

PipelineはJSONで書く必要があります。しかし文字列としてJSONを書くよりも、Dict型のオブジェクトを作成し、jsonモジュールでdumpsする方が扱い易く個人的には好みです。

In [57]:
sentence = \
    {
        "pipeline": [
            {
                "type": "readers.las",
                "filename": input_las
            }
        ]
    }

pipeline = pdal.Pipeline(json.dumps(sentence))
pipeline.execute()
rich.print(pipeline.metadata)

{
    'metadata': {
        'filters.merge': {},
        'readers.las': {
            'comp_spatialreference': '',
            'compressed': False,
            'copc': False,
            'count': 6684582,
            'creation_doy': 0,
            'creation_year': 0,
            'dataformat_id': 2,
            'dataoffset': 227,
            'filesource_id': 0,
            'global_encoding': 0,
            'global_encoding_base64': 'AAA=',
            'header_size': 227,
            'major_version': 1,
            'maxx': -3000,
            'maxy': 38250,
            'maxz': 1302.339999,
            'minor_version': 2,
            'minx': -4000,
            'miny': 37500,
            'minz': 142.85,
            'offset_x': -4000,
            'offset_y': 37500,
            'offset_z': 142.85,
            'point_length': 26,
            'project_id': '00000000-0000-0000-0000-000000000000',
            'scale_x': 4.6566128752458e-07,
            'scale_y': 3.49245965643435e-07,
            'scale_z': 5.39929606020451e-07,
            'software_id': 'TREND-POINT',
            'spatialreference': '',
            'srs': {
                'compoundwkt': '',
                'horizontal': '',
                'isgeocentric': False,
                'isgeographic': False,
                'json': {},
                'prettycompoundwkt': '',
                'prettywkt': '',
                'proj4': '',
                'units': {'horizontal': 'unknown', 'vertical': ''},
                'vertical': '',
                'wkt': ''
            },
            'system_id': 'FC'
        }
    }
}

## 座標参照系を定義する
---
上記でPrintされたものを見ると`spatialreference`が空になっているのが確認できます。
このままだとGISで使用しづらいので座標参照系を定義します。

In [66]:
pipe_json = \
    {
        "pipeline": [
            {
                "type": "readers.las",
                "filename": "../datasets/01ID7913_org.las"
            },
            {
                "type": "filters.reprojection",
                "in_srs": "EPSG:6669",
                "out_srs": "EPSG:6669",
            }
        ]
    }

pipeline = pdal.Pipeline(json.dumps(pipe_json))
pipeline.execute()
rich.print(pipeline.metadata)

{
    'metadata': {
        'filters.reprojection': {
            'comp_spatialreference': 'PROJCS["JGD2011 / Japan Plane Rectangular CS 
I",GEOGCS["JGD2011",DATUM["Japanese_Geodetic_Datum_2011",SPHEROID["GRS 
1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","1128"]],PRIMEM["Greenwich",0,AUTHORITY["EPS
G","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","6668"]],PROJECTION["Transv
erse_Mercator"],PARAMETER["latitude_of_origin",33],PARAMETER["central_meridian",129.5],PARAMETER["scale_factor",0.9
999],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Nort
hing",NORTH],AXIS["Easting",EAST],AUTHORITY["EPSG","6669"]]',
            'spatialreference': 'PROJCS["JGD2011 / Japan Plane Rectangular CS 
I",GEOGCS["JGD2011",DATUM["Japanese_Geodetic_Datum_2011",SPHEROID["GRS 
1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","1128"]],PRIMEM["Greenwich",0,AUTHORITY["EPS
G","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","6668"]],PROJECTION["Transv
erse_Mercator"],PARAMETER["latitude_of_origin",33],PARAMETER["central_meridian",129.5],PARAMETER["scale_factor",0.9
999],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Nort
hing",NORTH],AXIS["Easting",EAST],AUTHORITY["EPSG","6669"]]',
            'srs': {
                'compoundwkt': 'PROJCS["JGD2011 / Japan Plane Rectangular CS 
I",GEOGCS["JGD2011",DATUM["Japanese_Geodetic_Datum_2011",SPHEROID["GRS 
1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","1128"]],PRIMEM["Greenwich",0,AUTHORITY["EPS
G","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","6668"]],PROJECTION["Transv
erse_Mercator"],PARAMETER["latitude_of_origin",33],PARAMETER["central_meridian",129.5],PARAMETER["scale_factor",0.9
999],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Nort
hing",NORTH],AXIS["Easting",EAST],AUTHORITY["EPSG","6669"]]',
                'horizontal': 'PROJCS["JGD2011 / Japan Plane Rectangular CS 
I",GEOGCS["JGD2011",DATUM["Japanese_Geodetic_Datum_2011",SPHEROID["GRS 
1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","1128"]],PRIMEM["Greenwich",0,AUTHORITY["EPS
G","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","6668"]],PROJECTION["Transv
erse_Mercator"],PARAMETER["latitude_of_origin",33],PARAMETER["central_meridian",129.5],PARAMETER["scale_factor",0.9
999],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Nort
hing",NORTH],AXIS["Easting",EAST],AUTHORITY["EPSG","6669"]]',
                'isgeocentric': False,
                'isgeographic': False,
                'json': {
                    'type': 'ProjectedCRS',
                    'name': 'JGD2011 / Japan Plane Rectangular CS I',
                    'base_crs': {
                        'name': 'JGD2011',
                        'datum': {
                            'type': 'GeodeticReferenceFrame',
                            'name': 'Japanese Geodetic Datum 2011',
                            'ellipsoid': {
                                'name': 'GRS 1980',
                                'semi_major_axis': 6378137,
                                'inverse_flattening': 298.257222101
                            }
                        },
                        'coordinate_system': {
                            'subtype': 'ellipsoidal',
                            'axis': [
                                {
                                    'name': 'Geodetic latitude',
                                    'abbreviation': 'Lat',
                                    'direction': 'north',
                                    'unit': 'degree'
                                },
                                